In [17]:
import json
import pandas as pd
import numpy as np

In [18]:
file_dir = "C://Users/riley/documents/Data Analytics/Movies-ETL"
with open (f'{file_dir}/archive/wikipedia-movies.json', mode= 'r') as file:
    wikiMoviesRaw = json.load(file)

In [3]:
kaggleMetadata = pd.read_csv(f'{file_dir}/archive/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}/archive/ratings.csv')
wikiMoviesdf = pd.DataFrame(wikiMoviesRaw)

In [4]:
wikiMovies = [movie for movie in wikiMoviesRaw
             if("Director" in movie or "Directed by" in movie) and "imdb_link" in movie
             and "No. of episodes" not in movie]
len(wikiMovies)

7076

In [5]:
def cleanMovie(movie):
    movie = dict(movie)
    altTitles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            altTitles[key] = movie[key]
            movie.pop(key)
    if len(altTitles) > 0:
        movie['alt_titles'] = altTitles
    def changeColumnName(oldName, newName):
        if oldName in movie:
            movie[newName] = movie.pop(oldName)
        changeColumnName('Adaptation by', 'Writer(s)')
    changeColumnName('Country of origin', 'Country')
    changeColumnName('Directed by', 'Director')
    changeColumnName('Distributed by', 'Distributor')
    changeColumnName('Edited by', 'Editor(s)')
    changeColumnName('Length', 'Running time')
    changeColumnName('Original release', 'Release date')
    changeColumnName('Music by', 'Composer(s)')
    changeColumnName('Produced by', 'Producer(s)')
    changeColumnName('Producer', 'Producer(s)')
    changeColumnName('Productioncompanies ', 'Production company(s)')
    changeColumnName('Productioncompany ', 'Production company(s)')
    changeColumnName('Released', 'Release Date')
    changeColumnName('Release Date', 'Release date')
    changeColumnName('Screen story by', 'Writer(s)')
    changeColumnName('Screenplay by', 'Writer(s)')
    changeColumnName('Story by', 'Writer(s)')
    changeColumnName('Theme music composer', 'Composer(s)')
    changeColumnName('Written by', 'Writer(s)')
    
    return movie

In [ ]:
cleanMovies = [cleanMovie(movie) for movie in wikiMovies]
wikiMoviesdf = pd.DataFrame(cleanMovies)

In [ ]:
wikiMoviesdf['imdb_id'] = wikiMoviesdf['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wikiMoviesdf))
wikiMoviesdf.drop_duplicates(subset = 'imdb_id', inplace = True)
print(len(wikiMoviesdf))
wikiMoviesdf.head()

In [ ]:
wikiColumnstoKeep = [column for column in wikiMoviesdf.columns if wikiMoviesdf[column].isnull().sum() < len(wikiMoviesdf) * 0.9]
wikiMoviesdf = wikiMoviesdf[wikiColumnstoKeep]

In [ ]:
boxOffice = wikiMoviesdf['Box office'].dropna()

In [ ]:
boxOffice[boxOffice.map(lambda x: type(x) != str)]

In [ ]:
boxOffice = boxOffice.apply(lambda x: ' '.join(x) if type(x) == list else x)
import re

In [ ]:
formOne = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
formTwo = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
matchesFormOne = boxOffice.str.contains(formOne, flags=re.IGNORECASE, na = False).sum()

In [ ]:
matchesFormTwo = boxOffice.str.contains(formTwo, flags=re.IGNORECASE, na=False).sum()

In [ ]:
boxOffice[~matchesFormOne & ~matchesFormTwo]

In [ ]:
boxOffice = boxOffice.str.replace(r'\$.*[---](?![a-z])', '$', regex = True)

In [ ]:
boxOffice.str.extract(f'({formOne}|{formTwo})')

In [ ]:
def parseDollars(s):
    if type(s) != str:
        return np.nan

    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        value = float(s) * 10**6

        return value

    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        value = float(s) * 10**9

        return value

    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        s = re.sub('\$|,','', s)
        value = float(s)

        return value

    else:
        return np.nan

In [ ]:
wikiMoviesdf['box_office'] = boxOffice.str.extract(f'({formOne}|{formTwo})', flags=re.IGNORECASE)[0].apply(parseDollars)
wikiMoviesdf.drop('Box office', axis = 1, inplace = True)

In [ ]:
budget = wikiMoviesdf['Budget'].dropna()
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
matchesFormOne = budget.str.contains(formOne, flags=re.IGNORECASE, na=False)
matchesFormTwo = budget.str.contains(formTwo, flags=re.IGNORECASE, na=False)
budget[~matchesFormOne & ~matchesFormTwo]

In [ ]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matchesFormOne & ~matchesFormTwo]

In [ ]:
wikiMoviesdf['budget'] = budget.str.extract(f'({formOne}|{formTwo})', flags=re.IGNORECASE)[0].apply(parseDollars)
wikiMoviesdf.drop('Budget', axis = 1, inplace = True)

In [ ]:
releaseDate = wikiMoviesdf['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
dateFormOne = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
dateFormTwo = r'\d{4}.[01]\d.[0123]\d'
dateFormThree = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
dateFormFour = r'\d{4}'

In [ ]:
releaseData.str.extract(f'({dateFormOne}|{dateFormTwo}|{dateFormThree}|{dateFormFour})', flags=re.IGNORECASE)

In [ ]:
wikiMoviesdf['release_date'] = pd.to_datetime(releaseDate.str.extract(f'({dateFormOne}|{dateFormTwo}|{dateFormThree}|{dateFormFour})')[0], infer_datetime_format=True)

In [ ]:
runningTime = wikiMoviesdf['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
runningTime.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

In [ ]:
runningTime[runningTime.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

In [ ]:
runningTime.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()
runningTime[runningTime.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

In [ ]:
runningTimeExtract = runningTime.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
runningTimeExtract = runningTimeExtract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
wikiMoviesdf['running_time'] = runningTimeExtract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
wikiMoviesdf.drop('Running time', axis=1, inplace = True)

In [ ]:
kaggleMetadata['adult'].value_counts()
kaggleMetadata[~kaggleMetadata['adult'].isin(['True','False'])]

In [ ]:
kaggleMetadata = kaggleMetadata[kaggleMetadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
kaggleMetadata['video'] == 'True'

In [ ]:
kaggleMetadata['video'] = kaggleMetadata['video'] == 'True'
kaggleMetadata['budget'] = kaggleMetadata['budget'].astype(int)
kaggleMetadata['id'] = pd.to_numeric(kaggleMetadata['id'], errors='raise')
kaggleMetadata['popularity'] = pd.to_numeric(kaggleMetadata['popularity'], errors='raise')
kaggleMetadata['release_date'] = pd.to_datetime(kaggleMetadata['release_date'])

In [ ]:
ratings.info(null_counts=True)
pd.to_datetime(ratings['timestamp'], unit='s')
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()

In [ ]:
moviesdf = pd.merge(wikiMoviesdf, kaggleMetadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
moviesdf[['title_wiki', 'title_kaggle']]

In [ ]:
moviesdf[moviesdf['title_wiki'] != moviesdf['title_kaggle']][['title_wiki','title_kaggle']]
moviesdf[(moviesdf['title_kaggle'] == '')|(moviesdf['title_kaggle'].isnull())]

In [ ]:
moviesdf.fillna(0).plot(x='running_time', y='runtime', kind='scatter')

In [ ]:
moviesdf.fillna(0).plot(x='budget_wiki',y='budget_kaggle', kind='scatter')

In [ ]:
moviesdf.fillna(0).plot(x='box_office', y='revenue', kind='scatter')

In [ ]:
moviesdf.fillna(0)[moviesdf['box_office'] < 10**9].plot(x='box_office', y='revenue', kind='scatter')

In [ ]:
moviesdf[['release_date_wiki','release_date_kaggle']].plot(x='release_date_wiki', y='release_date_kaggle', style='.')

In [ ]:
moviesdf[(moviesdf['release_date_wiki'] > '1996-01-01') & (moviesdf['release_date_kaggle'] < '1965-01-01')].index

In [ ]:
moviesdf[moviesdf['release_date_wiki'].isnull()]

In [1]:
moviesdf['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)

NameError: name 'movies_df' is not defined

In [2]:
moviesdf['original_language'].value_counts(dropna=False)

NameError: name 'movies_df' is not defined

In [3]:
moviesdf[['Production company(s)','production_companies']]

NameError: name 'moviesdf' is not defined

In [4]:
moviesdf[['Production company(s)','production_companies']]

NameError: name 'moviesdf' is not defined

In [5]:
moviesdf.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

NameError: name 'moviesdf' is not defined

In [6]:
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)

In [7]:
fill_missing_kaggle_data(moviesdf, 'runtime', 'running_time')
fill_missing_kaggle_data(moviesdf, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(moviesdf, 'revenue', 'box_office')
moviesdf

NameError: name 'moviesdf' is not defined

In [8]:
for col in moviesdf.columns:
    lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
    value_counts = moviesdf[col].apply(lists_to_tuples).value_counts(dropna=False)
    num_values = len(value_counts)
    if num_values == 1:
        print(col)

NameError: name 'moviesdf' is not defined

In [10]:
moviesdf = moviesdf.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

NameError: name 'moviesdf' is not defined

In [11]:
moviesdf.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

NameError: name 'moviesdf' is not defined

In [13]:
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count().rename({'userId':'count'}, axis=1).pivot(index='movieId',columns='rating', values='count')

NameError: name 'ratings' is not defined

In [14]:
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

NameError: name 'rating_counts' is not defined

In [15]:
movies_with_ratings_df = pd.merge(moviesdf, rating_counts, left_on='kaggle_id', right_index=True, how='left')
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

NameError: name 'pd' is not defined